In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse

def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [6]:
A = np.array(nx.adjacency_matrix(g).todense(), dtype = np.float64)
deg_array = np.array(list(dict(g.degree).values()))

D_inv_A = np.zeros_like(A)

for i in range(num_nodes):
    D_inv_A[i] = A[i] / deg_array[i]


In [7]:
multi_hop_A = np.linalg.matrix_power(D_inv_A, 2)
# torch.save(multi_hop_A, "A_2hop.pkl")
# multi_hop_A = torch.load("A_2hop.pkl")

In [8]:
# set diagonal of 2-hop adjacency to 0
for i in range(num_nodes):
    multi_hop_A[i, i] = 0

In [9]:
node_list = list(g.nodes.keys())

In [10]:
def po_2hop_linear_model(graph, z_vec, alpha=1, beta=1, sigma=0.1, gamma=1, r1=1, r2=0.5):        
    y_vec = alpha + beta * z_vec + sigma * np.random.normal(size=(num_nodes, 1)) + gamma * (
    r1 * np.matmul(D_inv_A, z_vec) + r2 * np.matmul(multi_hop_A, z_vec)
)
    
    for i in range(num_nodes):
        graph.nodes[node_list[i]]["y"] = y_vec[i][0]
        graph.nodes[node_list[i]]["z"] = z_vec[i][0]

In [11]:
node_list = np.array(node_list)

# from node to index in node_list
node_list_index_dict = {}
for node in g.nodes:
    node_list_index_dict[node] = np.where(node_list == node)[0][0]

In [ ]:
# cluster-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

loss_list = np.zeros((repeat_num, len(ramps)))

for seed in tqdm(range(repeat_num)):
    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_cluster))   
    
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")
        torch.manual_seed(2)
        p_list = [0, ramps[num_step]]
        g_feat_list = []
        y_list = []
        
        for p in p_list:
            z_vector = np.zeros((num_nodes)) 
            tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
            if len(tr_clusters) > 0:
                tr_units = list(reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters]))
                for node in tr_units:
                    z_vector[node_list_index_dict[node]] = 1
                        
            po_2hop_linear_model(g, z_vector.reshape(-1,1)) # add dimension            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
            
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        loss_list[seed, num_step] = loss.item()
        
        
    torch.save(GATE_hat_list, "Result/incre_2hop_CR_cluster.pkl")
    # torch.save(loss_list, "2hop_loss_list.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.00, Loss: 8.1246
Epoch 0, Treat Prop: 0.02, Loss: 2.3953
Epoch 50, Treat Prop: 0.00, Loss: 0.1687
Epoch 50, Treat Prop: 0.02, Loss: 0.1879
Epoch 100, Treat Prop: 0.00, Loss: 0.0246
Epoch 100, Treat Prop: 0.02, Loss: 0.0445
Epoch 150, Treat Prop: 0.00, Loss: 0.0145
Epoch 150, Treat Prop: 0.02, Loss: 0.0277
Epoch 200, Treat Prop: 0.00, Loss: 0.0131
Epoch 200, Treat Prop: 0.02, Loss: 0.0219
Epoch 250, Treat Prop: 0.00, Loss: 0.0125
Epoch 250, Treat Prop: 0.02, Loss: 0.0181
Epoch 300, Treat Prop: 0.00, Loss: 0.0121
Epoch 300, Treat Prop: 0.02, Loss: 0.0161
Epoch 350, Treat Prop: 0.00, Loss: 0.0118
Epoch 350, Treat Prop: 0.02, Loss: 0.0148
0.014082062058150768
Seed: 0
Epoch 0, Treat Prop: 0.00, Loss: 8.1190
Epoch 0, Treat Prop: 0.05, Loss: 2.4123
Epoch 50, Treat Prop: 0.00, Loss: 0.1583
Epoch 50, Treat Prop: 0.05, Loss: 0.2304
Epoch 100, Treat Prop: 0.00, Loss: 0.0281
Epoch 100, Treat Prop: 0.05, Loss: 0.0799
Epoch 150, Treat Prop: 0.00, Loss: 0.0170
Epoch 150, Treat 

  0%|          | 1/1000 [00:23<6:30:18, 23.44s/it]

...

0.07845550030469894
Seed: 112
Epoch 0, Treat Prop: 0.00, Loss: 8.1186
Epoch 0, Treat Prop: 0.02, Loss: 2.4701
Epoch 50, Treat Prop: 0.00, Loss: 0.1665
Epoch 50, Treat Prop: 0.02, Loss: 0.2381
Epoch 100, Treat Prop: 0.00, Loss: 0.0251
Epoch 100, Treat Prop: 0.02, Loss: 0.0848
Epoch 150, Treat Prop: 0.00, Loss: 0.0165
Epoch 150, Treat Prop: 0.02, Loss: 0.0556
Epoch 200, Treat Prop: 0.00, Loss: 0.0147
Epoch 200, Treat Prop: 0.02, Loss: 0.0422
Epoch 250, Treat Prop: 0.00, Loss: 0.0138
Epoch 250, Treat Prop: 0.02, Loss: 0.0321
Epoch 300, Treat Prop: 0.00, Loss: 0.0129
Epoch 300, Treat Prop: 0.02, Loss: 0.0242
Epoch 350, Treat Prop: 0.00, Loss: 0.0127
Epoch 350, Treat Prop: 0.02, Loss: 0.0201
0.017787715420126915
Seed: 112
Epoch 0, Treat Prop: 0.00, Loss: 8.1201
Epoch 0, Treat Prop: 0.05, Loss: 2.4855
Epoch 50, Treat Prop: 0.00, Loss: 0.1527
Epoch 50, Treat Prop: 0.05, Loss: 0.2866
Epoch 100, Treat Prop: 0.00, Loss: 0.0295
Epoch 100, Treat Prop: 0.05, Loss: 0.1189
Epoch 150, Treat Prop: 0.00

 11%|█▏        | 113/1000 [39:08<5:05:05, 20.64s/it]

0.07725976407527924
Seed: 113
Epoch 0, Treat Prop: 0.00, Loss: 8.1150
Epoch 0, Treat Prop: 0.02, Loss: 2.3629
Epoch 50, Treat Prop: 0.00, Loss: 0.1598
Epoch 50, Treat Prop: 0.02, Loss: 0.1864
Epoch 100, Treat Prop: 0.00, Loss: 0.0254
Epoch 100, Treat Prop: 0.02, Loss: 0.0463
Epoch 150, Treat Prop: 0.00, Loss: 0.0143
Epoch 150, Treat Prop: 0.02, Loss: 0.0283
Epoch 200, Treat Prop: 0.00, Loss: 0.0132
Epoch 200, Treat Prop: 0.02, Loss: 0.0231
Epoch 250, Treat Prop: 0.00, Loss: 0.0128
Epoch 250, Treat Prop: 0.02, Loss: 0.0198
Epoch 300, Treat Prop: 0.00, Loss: 0.0125
Epoch 300, Treat Prop: 0.02, Loss: 0.0175
Epoch 350, Treat Prop: 0.00, Loss: 0.0123
Epoch 350, Treat Prop: 0.02, Loss: 0.0160
0.015093292109668255
Seed: 113
Epoch 0, Treat Prop: 0.00, Loss: 8.1135
Epoch 0, Treat Prop: 0.05, Loss: 2.3662
Epoch 50, Treat Prop: 0.00, Loss: 0.1563
Epoch 50, Treat Prop: 0.05, Loss: 0.2335
Epoch 100, Treat Prop: 0.00, Loss: 0.0293
Epoch 100, Treat Prop: 0.05, Loss: 0.0749
Epoch 150, Treat Prop: 0.00

 11%|█▏        | 114/1000 [39:28<5:01:31, 20.42s/it]

0.07535528391599655
Seed: 114
Epoch 0, Treat Prop: 0.00, Loss: 8.1082
Epoch 0, Treat Prop: 0.02, Loss: 2.3558
Epoch 50, Treat Prop: 0.00, Loss: 0.1607
Epoch 50, Treat Prop: 0.02, Loss: 0.1851
Epoch 100, Treat Prop: 0.00, Loss: 0.0232
Epoch 100, Treat Prop: 0.02, Loss: 0.0437
Epoch 150, Treat Prop: 0.00, Loss: 0.0143
Epoch 150, Treat Prop: 0.02, Loss: 0.0267
Epoch 200, Treat Prop: 0.00, Loss: 0.0130
Epoch 200, Treat Prop: 0.02, Loss: 0.0206
Epoch 250, Treat Prop: 0.00, Loss: 0.0124
Epoch 250, Treat Prop: 0.02, Loss: 0.0171
Epoch 300, Treat Prop: 0.00, Loss: 0.0121
Epoch 300, Treat Prop: 0.02, Loss: 0.0152
Epoch 350, Treat Prop: 0.00, Loss: 0.0118
Epoch 350, Treat Prop: 0.02, Loss: 0.0142
0.013473918661475182
Seed: 114
Epoch 0, Treat Prop: 0.00, Loss: 8.1241
Epoch 0, Treat Prop: 0.05, Loss: 2.4070
Epoch 50, Treat Prop: 0.00, Loss: 0.1529
Epoch 50, Treat Prop: 0.05, Loss: 0.2565
Epoch 100, Treat Prop: 0.00, Loss: 0.0295
Epoch 100, Treat Prop: 0.05, Loss: 0.0955
Epoch 150, Treat Prop: 0.00